# Writing the Data to a CSV 

In [ ]:
import csv
import codecs
#import cerberus
import re
import schema



OSM_PATH = "houston_texas.osm"

NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

SCHEMA = schema.schema


NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']



# Clean and shape node or way XML element to Python dict
def shape_element(element, node_attr_fields=NODE_FIELDS,
                  way_attr_fields=WAY_FIELDS, prob_ch=PROBLEMCHARS,
                  default_tag_type='regular'):
    
    tag_attribs = {}
    way_nodes = []
    tags = []
    count = 0
    if element.tag == 'node':
        tagfields = node_attr_fields
    elif element.tag == 'way':
        tagfields = way_attr_fields
    
    if element.tag == 'node' or 'way':
        for attrib in element.attrib:
            if attrib in tagfields:
                tag_attribs[attrib] = element.attrib[attrib]
                
                
                
    for subelem in element:
        if subelem.tag == 'tag' and prob_ch.match(subelem.attrib['k']) == None:
            tag = {}
            tag['id'] = tag_attribs['id']
            if subelem.attrib["k"] == 'addr:street':
                        tag["value"] = update_name(subelem.attrib["v"], mapping, regex )
            else:
                        tag["value"] = subelem.attrib["v"]
            key = subelem.attrib['k']
            tag['key'] = key[key.find(':') + 1:]
            if ':' in key:
                tag['type'] = key[:key.find(':')]
            else:
                tag['type'] = default_tag_type
            tags.append(tag)
        elif subelem.tag == 'nd':
            way_node = {}
            way_node['id'] = tag_attribs['id']
            way_node['node_id'] = subelem.attrib['ref']
            way_node['position'] = count
            count += 1
            way_nodes.append(way_node)

    if element.tag == 'node':
        return {'node': tag_attribs, 'node_tags': tags}
    elif element.tag == 'way':
        return {'way': tag_attribs, 'way_nodes': way_nodes, 'way_tags': tags}
    
# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(OSMFILE, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(OSMFILE, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()

def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_strings = (
            "{0}: {1}".format(k, v if isinstance(v, str) else ", ".join(v))
            for k, v in errors.iteritems()
        )
        raise cerberus.ValidationError(
            message_string.format(field, "\n".join(error_strings))
        )


# Extend csv.DictWriter to handle Unicode input
class UnicodeDictWriter(csv.DictWriter, object):
    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)
            

# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])

In [ ]:
process_map(OSM_PATH, validate=False)

### Chicking the files Size :

In [ ]:
import os
def file_size_mb(filePath): 
    return float(os.path.getsize(filePath)) /(1024 * 1024)

print 'The file size for houston_texas.osm is {} MB'.format(file_size_mb("houston_texas.osm"))
print 'The file size for h_db.db is {} MB'.format(file_size_mb("h_db.db"))
print 'The file size for nodes.csv  is {} MB'.format(file_size_mb("nodes.csv"))
print 'The file size for ways.csv is {} MB'.format(file_size_mb("ways.csv"))
print 'The file size for ways_nodes.csv is {} MB'.format(file_size_mb("ways_nodes.csv"))
print 'The file size for nodes_tags.csv is {} MB'.format(file_size_mb("nodes_tags.csv"))
print 'The file size for ways_tags.csv is {} MB'.format(file_size_mb("ways_tags.csv"))

### SQL queries

In [ ]:
import sqlite3
import pandas as pd

def query(query):
    conn = sqlite3.connect('h_db.db')
    data = pd.read_sql_query(query, conn)
    conn.close()
    return data

#### Number of tags in Nodes 

In [ ]:
query("SELECT COUNT(*) FROM nodes")

#### checking out the number of tags

In [2]:
query("SELECT COUNT(*) FROM ways")


####  How many unique users have created and updated this dataset ?

In [ ]:
query(
    '''
SELECT count(DISTINCT(uid)) AS "Unique Users"
FROM (SELECT uid FROM nodes
      UNION SELECT uid FROM ways) AS elements;
      '''
)

#### The top 5 streets that have the more node


In [ ]:
query( '''
SELECT ways_tags.value, COUNT(*) FROM ways_tags
WHERE ways_tags.key = 'name' AND ways_tags.type = 'regular' 
GROUP BY ways_tags.value ORDER BY COUNT(*) DESC LIMIT 6;
'''
)

#### The count of place of worships for christian is assure that Texas mybe somehowe a religious state,however as a muslime and lived there for few years I can easly found Mosque where I can go and pary.

In [ ]:
query ( '''
SELECT nodes_tags.value, COUNT(*) as Count
FROM nodes_tags 
JOIN
    (SELECT DISTINCT(id)
    FROM nodes_tags
    WHERE value='place_of_worship') as Sub
ON nodes_tags.id=Sub.id
WHERE nodes_tags.key='religion'
GROUP BY nodes_tags.value
ORDER BY Count DESC;
'''
   )

#### Who made the most of midification for houston map !?

In [ ]:
query (
    '''
SELECT DISTINCT nodes.user, COUNT(*)
FROM nodes
GROUP BY nodes.uid
ORDER BY COUNT(*) DESC
LIMIT 6;
'''
)

#### I used to be a customer of Bank of America,However it seems not the most puplor bank in the city :)


In [ ]:
sqldata (
    '''
    SELECT nodes_tags.value, COUNT(*) as num
        FROM nodes_tags
            JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value='bank') i
            ON nodes_tags.id=i.id
        WHERE nodes_tags.key='name'
        GROUP BY nodes_tags.value
        ORDER BY num DESC
        LIMIT 6;
        '''
)

# Additional Ideas to Improve OSM data

To improve OSM since it's an open source, it might be connected with other tools like Google Maps API espicilly that Google Maps API We have  more than one billion global monthly active users in over 200 countries, this amont of data will give  accurate real-time information for mapping around the world .

Also, another idea to improve the data set we could use  the popular game Pokemon Go to improve the dataset.Since the game got so popular and informations and that we can get out of can be really so helpfull to improve street mape .


## Conclusion and benefits as well as some anticipated problems .

First of all, after this project I truly believe the saying that the data analyst is spending almost %75 if his time in cleaning the data! Cleaning such data is not smooth as analysing cleaned dataset. Secondlly, if though I had lived in Houston for some time, after cleaning the street Map it seems now I know a lot more about the city which means how valube the data that is available online to get know something better.
By going all the way down in this process for auditing and cleaning the data, one benefits we can get from that, now we are sure and accurate about the information we have stated about the city of Houston. Cleaning the data can give deep information that not easily can be notice. Also it was interesting to me how many people have worked to improve and do some change in the street mapping. Some anticipated problems can come because it’s difficult to know exactly if a place should be an “area” or just a “way”, or another place should be an “street" or it's "Aven".So maybe in the future by clearly defining the "way" "area" "street" "aven" "road" that will help a lot improving the datasets.


